In [1]:
# importing packages
import folium
import pandas as pd

#Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [2]:
## Task 1: Mark all launch sites on a map
maps = folium.Map(Locations = [(28.562302, -80.577356), (28.563197, -80.576820), 
                               (28.573255, -80.646895), (34.632834, -120.610745)])
maps

In [3]:
import requests

import io

# URL of the CSV file
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

# Fetch the content from the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Read the content into a pandas DataFrame
    data = pd.read_csv(io.StringIO(response.text))
    # Display the first few rows of the DataFrame
    print(data.head())
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")

   Flight Number        Date Time (UTC) Booster Version  Launch Site  \
0              1  2010-06-04   18:45:00  F9 v1.0  B0003  CCAFS LC-40   
1              2  2010-12-08   15:43:00  F9 v1.0  B0004  CCAFS LC-40   
2              3  2012-05-22    7:44:00  F9 v1.0  B0005  CCAFS LC-40   
3              4  2012-10-08    0:35:00  F9 v1.0  B0006  CCAFS LC-40   
4              5  2013-03-01   15:10:00  F9 v1.0  B0007  CCAFS LC-40   

                                             Payload  Payload Mass (kg)  \
0               Dragon Spacecraft Qualification Unit                0.0   
1  Dragon demo flight C1, two CubeSats,  barrel o...                0.0   
2                             Dragon demo flight C2+              525.0   
3                                       SpaceX CRS-1              500.0   
4                                       SpaceX CRS-2              677.0   

       Orbit         Customer        Landing Outcome  class        Lat  \
0        LEO           SpaceX  Failure   (

In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = data[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [6]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map

An example of folium.Marker:

`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [12]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [14]:
# Initiate the map
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
for i in range(0, len(launch_sites_df)):
    folium.Circle(
      location=[launch_sites_df.iloc[i]['Lat'], launch_sites_df.iloc[i]['Long']],
      radius=1000,
      color='crimson',
      fill=True,
      fill_color='crimson'
   ).add_to(site_map)
    folium.Marker([launch_sites_df.iloc[i]['Lat'], launch_sites_df.iloc[i]['Long']], popup=launch_sites_df.iloc[i]['Launch Site']).add_to(site_map)

Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [11]:
# Task 2: Mark the success/failed launches for each site on the map
# Create a new dataframe to show the number of failed & successful launches for each site
spacex_df = data[['Launch Site', 'class']].groupby('Launch Site').agg(['sum', 'count'])
spacex_df.columns = spacex_df.columns.droplevel()
spacex_df.reset_index(inplace=True)
spacex_df.columns = ['Launch Site', 'Success', 'Total']
spacex_df['Failed'] = spacex_df['Total'] - spacex_df['Success']
spacex_df

,Launch Site,Success,Total,Failed
0,CCAFS LC-40,7,26,19
1,CCAFS SLC-40,3,7,4
2,KSC LC-39A,10,13,3
3,VAFB SLC-4E,4,10,6


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [ ]:
marker_cluster = MarkerCluster()

*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value

In [ ]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    

*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [ ]:
# Ensure marker_cluster is defined
marker_cluster = MarkerCluster()

# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# Define the assign_marker_color function
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'

# for each row in data, add a Marker object to the MarkerCluster
# and specify coordinates [Lat, Long], icon color, icon glyph, and popup label
for index, record in data.iterrows():
    marker = folium.Marker([record['Lat'], record['Long']], 
                           icon=folium.Icon(color=assign_marker_color(record['class'])))
    marker_cluster.add_child(marker)

site_map    # Display the map

Your updated map may look like the following screenshots:

From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.

In [23]:
# Create a folium map object
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

Next, we need to explore and analyze the proximities of launch sites.

Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [24]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [25]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.

In [32]:
# find coordinate of the closest coastline
coastline_lat = 28.56342
coastline_lon = -80.57188

# Define the launch site coordinates (example: CCAFS LC-40)
launch_site_lat = launch_sites_df.loc[launch_sites_df['Launch Site'] == 'CCAFS LC-40', 'Lat'].values[0]
launch_site_lon = launch_sites_df.loc[launch_sites_df['Launch Site'] == 'CCAFS LC-40', 'Long'].values[0]

distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print(f'The distance between CCAFS LC-40 and the closest coastline is {round(distance_coastline, 2)} km')


The distance between CCAFS LC-40 and the closest coastline is 0.55 km


In [33]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )
# site_map.add_child(distance_marker)
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
        )
    )
site_map.add_child(distance_marker)
site_map


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [34]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# Add the line to the site_map
lines=folium.PolyLine(locations=[[coastline_lat, coastline_lon],[launch_site_lat, launch_site_lon]], weight=1)
site_map.add_child(lines)
site_map

Your updated map with distance line should look like the following screenshot:
<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first

In [36]:
# Draw a line between a launch site to its closest city, railway, highway, etc...
# You need to use MousePosition to get the coordinate (Lat, Long) for a mouse over on the map
# The folium click event handler cannot be used to get coordinates as the coordinates are fixed
# Add a mouse position event to the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)


A railway map symbol may look like this:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>

A city map symbol may look like this:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [37]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
closest_city_lat = 28.3922  # Example latitude for the closest city
closest_city_lon = -80.6077  # Example longitude for the closest city

# Calculate the distance to the closest city
distance_city = calculate_distance(launch_site_lat, launch_site_lon, closest_city_lat, closest_city_lon)

# Create a marker for the closest city
city_marker = folium.Marker(
    [closest_city_lat, closest_city_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),
        )
    )
site_map.add_child(city_marker)

# Draw a line between the launch site and the closest city
city_line = folium.PolyLine(locations=[[closest_city_lat, closest_city_lon], [launch_site_lat, launch_site_lon]], weight=1)
site_map.add_child(city_line)

site_map


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.